In [1]:
import numpy as np
import IM_rank_correlations as IMR

In [24]:
ranks_matrix = IMR.ranks(IMR.scores_matrix, IMR.measures_arr);
measures_arr = ranks_matrix.measures_arr;
measures_dict = ranks_matrix.measures_dict;

In [25]:
ranks_matrix.compute_correlation()

In [26]:
def find_redundant(measures_arr):
#     print(len(measures_arr))
    for i in range(len(measures_arr)):
        for j in range(len(measures_arr)):
            if (i > j):
                if (np.around(ranks_matrix.corr_spearman[i][j], 3) == 1):
#                     print(measures_arr[i],measures_arr[j]);
                    return(j);

# find_redundant(measures_arr)

while find_redundant(measures_arr):
    i = find_redundant(measures_arr);
    measures_dict, measures_arr = ranks_matrix.remove_outliers(i);

In [ ]:
def find_outlier(clusters):
    arr = []
    for i in range(len(clusters)):
        if len(clusters[i]) == 1:
            arr.append(clusters[i]);
    return np.array(arr);


# while len(find_outlier(clusters)):
#     idx_outliers = find_outlier(clusters);
#     print(measures_arr[idx_outliers]);
#     measures_dict, measures_arr = ranks_matrix.remove_outliers(idx_outliers);
#     clusters = ranks_matrix.form_clusters(n_clusters);

In [35]:
n_clusters = 10;
clusters = ranks_matrix.form_clusters(n_clusters);
for cluster in clusters:
    print(len(cluster), cluster);
    print(measures_arr[cluster])

4 [ 0  4  6 12]
['recall' 'support' 'kulczynsky_1' 'cosine']
16 [ 1  3  9 10 11 16 17 19 20 21 22 23 28 29 32 34]
['mutual_information' 'accuracy' 'relative_risk' 'novelty' 'yules_y'
 'conviction' 'information_gain' 'klosgen' 'zhang'
 'normalized_mutual_information' 'one_way_support' 'two_way_support'
 'kappa' 'putative_causal_dependency' 'collective_strength' 'dependency']
1 [2]
['negative_reliability']
7 [ 5 13 14 15 18 30 31]
['confidence_causal' 'least_contradiction' 'confirm_descriptive'
 'confirm_causal' 'laplace_correction' 'example_counterexample_rate'
 'confirmed_confidence_causal']
1 [7]
['coverage']
1 [8]
['prevalence']
1 [24]
['implication_index']
2 [25 33]
['gini_index' 'j_measure']
1 [26]
['goodman_kruskal']
1 [27]
['leverage']


In [28]:
len(ranks_matrix.measures_arr)

35

In [6]:
#removing all measures except the ones in cluster 1 (16 measures)
to_remove = np.empty(0, int);

for cluster in clusters:
    if not (len(cluster) == 16):
        to_remove = np.append(to_remove, cluster);

measures_dict, measures_arr = ranks_matrix.remove_outliers(to_remove);

In [36]:
misc_cluster = np.array([], int);
misc_cluster = np.append(misc_cluster, clusters[2]);
misc_cluster = np.append(misc_cluster, clusters[4]);
misc_cluster = np.append(misc_cluster, clusters[5]);
misc_cluster = np.append(misc_cluster, clusters[6]);
misc_cluster = np.append(misc_cluster, clusters[7]);
misc_cluster = np.append(misc_cluster, clusters[8]);
misc_cluster = np.append(misc_cluster, clusters[9]);

clusters_new = [];
clusters_new.append(clusters[0]);
clusters_new.append(clusters[1]);
clusters_new.append(clusters[3]);
clusters_new.append(misc_cluster);

clusters_new = np.array(clusters_new);
clusters = clusters_new;

# print(clusters_new)
print([measures_arr[clusters_new[i]] for i in range(len(clusters_new))])

[array(['recall', 'support', 'kulczynsky_1', 'cosine'], 
      dtype='<U29'), array(['mutual_information', 'accuracy', 'relative_risk', 'novelty',
       'yules_y', 'conviction', 'information_gain', 'klosgen', 'zhang',
       'normalized_mutual_information', 'one_way_support',
       'two_way_support', 'kappa', 'putative_causal_dependency',
       'collective_strength', 'dependency'], 
      dtype='<U29'), array(['confidence_causal', 'least_contradiction', 'confirm_descriptive',
       'confirm_causal', 'laplace_correction',
       'example_counterexample_rate', 'confirmed_confidence_causal'], 
      dtype='<U29'), array(['negative_reliability', 'coverage', 'prevalence',
       'implication_index', 'gini_index', 'j_measure', 'goodman_kruskal',
       'leverage'], 
      dtype='<U29')]


In [37]:
for cluster in clusters:
    print(len(cluster), cluster);
    print(measures_arr[cluster])

4 [ 0  4  6 12]
['recall' 'support' 'kulczynsky_1' 'cosine']
16 [ 1  3  9 10 11 16 17 19 20 21 22 23 28 29 32 34]
['mutual_information' 'accuracy' 'relative_risk' 'novelty' 'yules_y'
 'conviction' 'information_gain' 'klosgen' 'zhang'
 'normalized_mutual_information' 'one_way_support' 'two_way_support'
 'kappa' 'putative_causal_dependency' 'collective_strength' 'dependency']
7 [ 5 13 14 15 18 30 31]
['confidence_causal' 'least_contradiction' 'confirm_descriptive'
 'confirm_causal' 'laplace_correction' 'example_counterexample_rate'
 'confirmed_confidence_causal']
8 [ 2  7  8 24 25 33 26 27]
['negative_reliability' 'coverage' 'prevalence' 'implication_index'
 'gini_index' 'j_measure' 'goodman_kruskal' 'leverage']


In [18]:
import measure_classification as mc;
import compute_invariance as compute_invariance;


In [38]:
property_dict = compute_invariance.compute_property_vectors(measures_dict);
X,Y = mc.initialize_X_Y(measures_arr, property_dict);

In [39]:
X = mc.form_X(property_dict, measures_dict, X);
Y = mc.assign_clusters_to_Y(clusters, Y);
X.shape

(35, 805)

In [40]:
# import scipy.spatial.distance as ssd
# import scipy.misc as sm;
# import numpy as np;

# for c in range(len(clusters)):
#     n= sm.comb(len(clusters[c]),2);
#     arr = np.zeros(n, float);
# #     print(c)
#     k=0;
#     for i in clusters[c]:
#         for j in clusters[c]:
#             if (i > j):
# #                 print(measures_arr[i], measures_arr[j]);
# #                 print(X[i], X[j])
#                 arr[k] = ssd.hamming(X[i], X[j]);
#                 k += 1;
#     print(np.average(arr))
# # print(measures_arr[clusters[0][1]], measures_arr[clusters[0][2]])
# # ssd.jaccard(X[clusters[0][1]], X[clusters[0][4]])

In [41]:
predictions, tree_classifier = mc.classify_decision_tree(X,Y, criterion='entropy');
print(tree_classifier.score(X,Y))

0.742857142857


In [34]:
mc.export_to_pdf(tree_classifier, n_clusters, len(measures_arr))

In [ ]:
# measures_arr[(X[:,0] == 0) & ((X[:,1] == 0)) & ((X[:,2] == 0)) & ((X[:,4] == 0)) & ((X[:,5] == 0)) & ((X[:,6] == 0)) & ((X[:,7] == 0))]
measures_arr[(X[:,36] == 1) & ((X[:,48] == -1)) & ((X[:,111] == -1))]

In [ ]:
# np.around(ranks_matrix.corr_spearman[measures_dict['example_counterexample_rate'], measures_dict['laplace_correction']], 3)

# X = np.delete(X, 3, axis=1)

In [ ]:
# ranks_matrix.show_dendrogram()

In [52]:
# a = np.empty((len(X[0,:]),len(clusters)), float);
# for p in range(len(clusters)):
#     for i in range(len(X[0,:])):
# #         a[i][p] = np.sum(X[clusters[p],i])/len(clusters[p]);
#         a[i][p] = np.sum(X[clusters[p],i]);
#         print(a[i][p])
# #     print('----')
# # a = (1 + a)/2;

16.0
0.0


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=[15,30], ncols=1, nrows=1);
plt.imshow(a, interpolation='none', cmap=plt.cm.RdBu);
# plt.colorbar();
plt.show()

In [ ]:
import scipy.misc as sm

In [ ]:
7 + (2 * sm.comb(7,2)) + (4 * sm.comb(7,3)) + (8 * sm.comb(7,4)) + (16 * sm.comb(7,5))

In [ ]:
7 + (2 * sm.comb(7,2)) + (4 * sm.comb(7,3)) + (8 * sm.comb(7,4))

In [ ]:
import itertools
inv = ['P2', 'P3', 'O1', 'O2', 'O3', 'O4', 'O5'];
for i,j,l,m,n in itertools.combinations(range(7),5):
    print(inv[i] + ' and ' + inv[j] + ' and ' + inv[l] + ' and ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' and ' + inv[l] + ' and ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' and ' + inv[j] + ' or ' + inv[l] + ' and ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' or ' + inv[l] + ' and ' + inv[m] + ' and ' + inv[n]);

    print(inv[i] + ' and ' + inv[j] + ' and ' + inv[l] + ' or ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' and ' + inv[l] + ' or ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' and ' + inv[j] + ' or ' + inv[l] + ' or ' + inv[m] + ' and ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' or ' + inv[l] + ' or ' + inv[m] + ' and ' + inv[n]);

    print(inv[i] + ' and ' + inv[j] + ' and ' + inv[l] + ' and ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' and ' + inv[l] + ' and ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' and ' + inv[j] + ' or ' + inv[l] + ' and ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' or ' + inv[l] + ' and ' + inv[m] + ' or ' + inv[n]);

    print(inv[i] + ' and ' + inv[j] + ' and ' + inv[l] + ' or ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' and ' + inv[l] + ' or ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' and ' + inv[j] + ' or ' + inv[l] + ' or ' + inv[m] + ' or ' + inv[n]);
    print(inv[i] + ' or ' + inv[j] + ' or ' + inv[l] + ' or ' + inv[m] + ' or ' + inv[n]);

